# 문서 벡터화 Document Vectorization
- BOW
- TF-IDF
- DTM / TDM

## BOW Bag of Words
CountVectorizer클래스를 통해 텍스트를 토큰화하고, 단어빈도수 기반으로 특성벡터를 생성

In [1]:
sentences = [
    'I love my dog.',
    'I love my cat.',
    'I love my dog and love my cat.',
    'You love my dog!',
    'Do you think my dog is amazing?'
]

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

# 객체 생성
vectorizer = CountVectorizer() # 객체 생성
features = vectorizer.fit_transform(sentences)
features

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 22 stored elements and shape (5, 10)>

In [3]:
# 값 확인
features.toarray() # 희소행렬을 Numpy 2차원배열(밀집 배열 = dense array)로 바꿔서 확인

array([[0, 0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 1, 0, 0],
       [0, 1, 1, 0, 1, 0, 2, 2, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 0, 1, 1, 1, 0, 1, 1, 1]])

In [4]:
feature_names = vectorizer.get_feature_names_out() # 학습된 vocabulary의 피처명 목록
feature_names

array(['amazing', 'and', 'cat', 'do', 'dog', 'is', 'love', 'my', 'think',
       'you'], dtype=object)

In [5]:
import pandas as pd

bow_df = pd.DataFrame(
    features.toarray(),      # 희소행렬을 dense 배열로 바꿔서 생성
    columns = feature_names, # 단어사전(피처명)
    index = ['sent1', 'sent2', 'sent3', 'sent4', 'sent5' ] # 행 이름 지정
)
bow_df

,amazing,and,cat,do,dog,is,love,my,think,you
sent1,0,0,0,0,1,0,1,1,0,0
sent2,0,0,1,0,0,0,1,1,0,0
sent3,0,1,1,0,1,0,2,2,0,0
sent4,0,0,0,0,1,0,1,1,0,1
sent5,1,0,0,1,1,1,0,1,1,1


features가 커지면 toarray()로 바꿀시 메모리를 많이 사용해서, 큰 데이터에서는 이룹만 확인하거나 희소형태를 유지하는 방식을 사용한다.

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

bow_sim = cosine_similarity(bow_df)
bow_sim_df = pd.DataFrame(
    bow_sim,             # 유사도 결과를 데이터로 사용
    columns = sentences, # 열은 문장(비교 대상)
    index = sentences    # 행은 문장 (기준)
)

bow_sim_df

,I love my dog.,I love my cat.,I love my dog and love my cat.,You love my dog!,Do you think my dog is amazing?
I love my dog.,1.000000,0.666667,0.870388,0.866025,0.436436
I love my cat.,0.666667,1.000000,0.870388,0.577350,0.218218
I love my dog and love my cat.,0.870388,0.870388,1.000000,0.753778,0.341882
You love my dog!,0.866025,0.577350,0.753778,1.000000,0.566947
Do you think my dog is amazing?,0.436436,0.218218,0.341882,0.566947,1.000000


## DTM | TDM
- DTM Document-Term Matrix 문서별 용어 행렬
- TDM Term-Document Matrix 용어별 문서 행렬

In [7]:
# DTM : BoW 결과를 문서-단어 행렬(DataFrame)로 구성
dtm = pd.DataFrame(
    features.toarray(),
    columns = feature_names,
    index = ['sent1', 'sent2', 'sent3', 'sent4', 'sent5' ]
)
dtm

,amazing,and,cat,do,dog,is,love,my,think,you
sent1,0,0,0,0,1,0,1,1,0,0
sent2,0,0,1,0,0,0,1,1,0,0
sent3,0,1,1,0,1,0,2,2,0,0
sent4,0,0,0,0,1,0,1,1,0,1
sent5,1,0,0,1,1,1,0,1,1,1


In [8]:
# TDM : DTM을 전치해서 단어-문서 행렬로 변환. 단어를 기준으로 문서별 빈도를 비교/분석
tdm = dtm.transpose()  # (문서 x 단어) -> (단어 x 문서) TDM으로 전치
tdm

,sent1,sent2,sent3,sent4,sent5
amazing,0,0,0,0,1
and,0,0,1,0,0
cat,0,1,1,0,0
do,0,0,0,0,1
dog,1,0,1,1,1
is,0,0,0,0,1
love,1,1,2,1,0
my,1,1,2,1,1
think,0,0,0,0,1
you,0,0,0,1,1


## TF-IDF
`TfidfVectorizer`의 계산은 TF-IDF(Term Frequency-Inverse Document Frequency)라는 지표를 사용한다.

이는 각 단어의 중요도를 고려하여 문서 내에서의 가중치를 계산하는 방식이다.

**용어**
- $tf(t, d)$: 특정 단어 $t$가 문서 $d$에서 등장한 횟수 (Term Frequency)
- $df(t)$: 특정 단어 $t$가 등장한 문서의 수 (Document Frequency)
- $N$: 전체 문서의 수

**TF (Term Frequency)**
단어 $t$의 문서 $d$에서의 빈도를 계산하는데, 가장 일반적인 방법은 해당 단어의 단순 빈도로 정의한다.

$
tf(t, d) = \frac{\text{단어 } t \text{의 문서 } d \text{ 내 등장 횟수}}{\text{문서 } d \text{의 전체 단어 수}}
$

**IDF (Inverse Document Frequency)**
단어가 전체 문서에서 얼마나 중요한지를 계산한다. 특정 단어가 많은 문서에서 등장하면, 이 단어는 중요도가 낮아진다. 이를 반영하기 위해 아래와 같은 식을 사용한다:

$
idf(t) = \log\left(\frac{1 + N}{1 + df(t)}\right) + 1
$

여기서 $1$을 더하는 이유는, 특정 단어가 모든 문서에 등장하지 않을 경우 $df(t) = 0$이 되어, 분모가 $0$이 되는 것을 방지하기 위함이다.

예를 들어, $\log(5/(1+1))$과 $\log(5/(1+2))$를 계산하면, 각각 $0.3979$와 $0.2218$이 된다.

**TF-IDF 계산**
위의 TF와 IDF를 결합하여 TF-IDF 가중치를 계산한다:

$
\text{tf-idf}(t, d) = tf(t, d) \times idf(t)
$

**TfidfVectorizer의 주요 파라미터**
<table border="1" cellpadding="5" cellspacing="0">
  <tr style="background-color: #f2f2f2;">
    <th>Parameter</th>
    <th>Description</th>
    <th>Default Value</th>
  </tr>
  <tr style="background-color: #675202;">
    <td><b>max_df</b></td>
    <td>문서의 비율 값으로서, 해당 비율 이상 나타나는 단어를 무시한다. <br> 예를 들어, max_df=0.8이면, 80% 이상의 문서에서 나타나는 단어는 제외된다.</td>
    <td>1.0</td>
  </tr>
  <tr style="background-color: #675202;">
    <td><b>min_df</b></td>
    <td>문서의 비율 값 또는 정수로, 해당 비율 이하 나타나는 단어를 무시한다. <br> 예를 들어, min_df=2이면, 두 개 이하의 문서에서만 나타나는 단어는 제외된다.</td>
    <td>1</td>
  </tr>
  <tr style="background-color: #675202;">
    <td><b>ngram_range</b></td>
    <td>(min_n, max_n) 형식으로, 사용할 n-gram의 범위를 정의한다. <br> 예를 들어, (1, 2)로 설정하면 unigram과 bigram을 고려한다.</td>
    <td>(1, 1)</td>
  </tr>
  <tr style="background-color: #675202;">
    <td>stop_words</td>
    <td>불용어를 지정할 수 있다. "english"로 설정하면 영어 불용어를 사용한다.</td>
    <td>None</td>
  </tr>
  <tr>
    <td>max_features</td>
    <td>벡터화할 때 고려할 최대 단어 수를 설정한다. 빈도순으로 상위 단어들이 선택된다.</td>
    <td>None</td>
  </tr>
  <tr>
    <td>use_idf</td>
    <td>IDF(역문서 빈도)를 사용할지 여부를 지정한다. False로 설정하면 단순히 TF 값만 사용한다.</td>
    <td>True</td>
  </tr>
  <tr>
    <td>smooth_idf</td>
    <td>IDF 계산 시, 0으로 나누는 것을 피하기 위해 추가적인 smoothing을 수행한다.</td>
    <td>True</td>
  </tr>
  <tr>
    <td>sublinear_tf</td>
    <td>TF 값에 대해 sublinear scaling (1 + log(tf))를 적용할지 지정한다.</td>
    <td>False</td>
  </tr>
</table>

In [9]:
# TF-IDF 백터라이저 : 단어 빈도를 중요도(TF-IDF)로 가중한 문서-단어 행렬 생성
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
features = tfidf_vectorizer.fit_transform(sentences) # 어휘 학습 + TF-IDf 희소행렬로 변환
print(features)

features = features.toarray() # dense 배열로 변환
features

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 22 stored elements and shape (5, 10)>
  Coords	Values
  (0, 6)	0.6068561362933035
  (0, 7)	0.5132750331803866
  (0, 4)	0.6068561362933035
  (1, 6)	0.5152898800248592
  (1, 7)	0.43582888010783327
  (1, 2)	0.7379224395611763
  (2, 6)	0.5533643125368353
  (2, 7)	0.4680319912607929
  (2, 4)	0.27668215626841763
  (2, 2)	0.3962235232075343
  (2, 1)	0.4911088441748528
  (3, 6)	0.4580537876334307
  (3, 7)	0.3874189597587254
  (3, 4)	0.4580537876334307
  (3, 9)	0.6559573194109529
  (4, 7)	0.20905444571530132
  (4, 4)	0.24716957771281237
  (4, 9)	0.3539599453463846
  (4, 3)	0.4387242287788317
  (4, 8)	0.4387242287788317
  (4, 5)	0.4387242287788317
  (4, 0)	0.4387242287788317


array([[0.        , 0.        , 0.        , 0.        , 0.60685614,
        0.        , 0.60685614, 0.51327503, 0.        , 0.        ],
       [0.        , 0.        , 0.73792244, 0.        , 0.        ,
        0.        , 0.51528988, 0.43582888, 0.        , 0.        ],
       [0.        , 0.49110884, 0.39622352, 0.        , 0.27668216,
        0.        , 0.55336431, 0.46803199, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.45805379,
        0.        , 0.45805379, 0.38741896, 0.        , 0.65595732],
       [0.43872423, 0.        , 0.        , 0.43872423, 0.24716958,
        0.43872423, 0.        , 0.20905445, 0.43872423, 0.35395995]])

CountVectorizer와 비교했을 때 TF-IDF는 자주 나오지만 흔한 단어의 영향은 줄이고, 특정 문서에만 특징적인 단어는 더 강조해준다.

In [10]:
feature_names = tfidf_vectorizer.get_feature_names_out() # TF-IDF 벡터의 열에 대응하는 토큰 목록
print(feature_names)

['amazing' 'and' 'cat' 'do' 'dog' 'is' 'love' 'my' 'think' 'you']


In [11]:
# Bag of Words
sent_df = pd.DataFrame(
    features,
    columns= feature_names,
    index = ['sent1', 'sent2', 'sent3', 'sent4', 'sent5' ]
)
sent_df

,amazing,and,cat,do,dog,is,love,my,think,you
sent1,0.000000,0.000000,0.000000,0.000000,0.606856,0.000000,0.606856,0.513275,0.000000,0.000000
sent2,0.000000,0.000000,0.737922,0.000000,0.000000,0.000000,0.515290,0.435829,0.000000,0.000000
sent3,0.000000,0.491109,0.396224,0.000000,0.276682,0.000000,0.553364,0.468032,0.000000,0.000000
sent4,0.000000,0.000000,0.000000,0.000000,0.458054,0.000000,0.458054,0.387419,0.000000,0.655957
sent5,0.438724,0.000000,0.000000,0.438724,0.247170,0.438724,0.000000,0.209054,0.438724,0.353960


TypeError: DataFrame.sort_values() missing 1 required positional argument: 'by'

In [14]:
# tf-idf 도출과정
import numpy as np

sent = sent_df.iloc[1] # 두 번째 문장(sent2)의 TF-IDF 벡터 행 추출
sent

amazing    0.000000
and        0.000000
cat        0.737922
do         0.000000
dog        0.000000
is         0.000000
love       0.515290
my         0.435829
think      0.000000
you        0.000000
Name: sent2, dtype: float64

In [15]:
# I love my cat
n_doc_terms = 4  # 이 문서의 총 토큰수 (문서 길이)
n_docs = 5 + 1   # 전체 문서 수 (+1은 스무딩 / 분모 안정화)

love_tf = 1 / n_doc_terms               # 해당 단어 빈도 +1 / 문서 내 총 토큰 수
love_idf = np.log(n_docs / (1 + 4)) + 1 # log(N / (1 + love가 등장한 문서 수) + 1 
love_tfidf = love_tf * love_idf         # TF-IDF = TF * IDF
print(love_tfidf)

my_tf = 1 / n_doc_terms                 # 해당 단어 빈도 +1 / 문서 내 총 토큰 수
my_idf = np.log(n_docs / (1 + 4)) + 1   # log(N / (1 + my가 등장한 문서 수) + 1 
my_tfidf = my_tf * my_idf               # TF-IDF = TF * IDF
print(my_tfidf)

cat_tf = 1 / n_doc_terms
cat_idf = np.log(n_docs / (1 + 4)) + 1
cat_tfidf = cat_tf * cat_idf
print(cat_tfidf)

# 벡터 정규화
sent2_vecs = np.array([0, 0, cat_tfidf, 0, 0, 0, love_tfidf, my_tfidf, 0, 0]) # sent2의 TF-IDF 벡터
norm = np.linalg.norm(sent2_vecs) # L2 노름 (벡터의 크기) 계산
sent2_vecs = sent2_vecs / norm    # 각 요소를 벡터 전체 크기로 나눠 단위벡터로 정규화
sent2_vecs

0.29558038919848867
0.29558038919848867
0.29558038919848867


array([0.        , 0.        , 0.57735027, 0.        , 0.        ,
       0.        , 0.57735027, 0.57735027, 0.        , 0.        ])

In [ ]:
# TF-IDF 기반 문장 유사도 : 코사인 유사도로 문장 간 유사도 행렬 계산
from sklearn.metrics.pairwise import cosine_similarity

sent_sim = cosine_similarity(sent_df)
sent_sim_df = pd.DataFrame(
    sent_sim,             # 데이터는 유사도 결과
    columns = sentences,  # 비교 대상 문장 
    index = sentences     # 기준 문장
)

sent_sim_df

,I love my dog.,I love my cat.,I love my dog and love my cat.,You love my dog!,Do you think my dog is amazing?
I love my dog.,1.000000,0.536407,0.743948,0.754798,0.257299
I love my cat.,0.536407,1.000000,0.781507,0.404879,0.091112
I love my dog and love my cat.,0.743948,0.781507,1.000000,0.561530,0.166232
You love my dog!,0.754798,0.404879,0.561530,1.000000,0.426391
Do you think my dog is amazing?,0.257299,0.091112,0.166232,0.426391,1.000000


TF-IDF는 BoW보다 흔한 단어의 영향이 줄어, 특징 단어 중심으로 유사도가 계산된다.

## 자연어 임베딩(Embedding)이란?

자연어 임베딩은 **텍스트(단어/문장/문서)를 고정 길이의 실수 벡터(vector)로 바꾸는 표현 방식**이다.  
즉, 컴퓨터가 다룰 수 있도록 **문자를 수치화**하되, 단순 ID가 아니라 **의미·문맥·문법적 특징**이 벡터 공간에 반영되도록 만든다.

- 임베딩 덕분에 텍스트끼리 **유사도(코사인 유사도 등)**를 계산할 수 있고  
- 머신러닝/딥러닝 모델에 **입력 피처**로 넣을 수 있으며  
- “king - man + woman ≈ queen” 같은 **의미 연산**이 벡터 공간에서 가능해진다.

---

## 임베딩 핵심 정리 표

| 구분 | 내용 | 예시/포인트 |
|---|---|---|
| 정의 | 텍스트를 **고정 길이 실수 벡터**로 변환 | 단어/문장/문서 → `R^d` 벡터 |
| 목적 | 컴퓨터가 텍스트를 수치로 처리 + 의미를 보존 | 단순 라벨 인코딩과 다름 |
| 표현 대상 | 단어 임베딩 / 문장 임베딩 / 문서 임베딩 | word2vec(단어), SBERT(문장) |
| 벡터 의미 | 벡터 공간에서 **거리/각도**가 의미적 유사성을 반영 | 가까울수록 의미가 비슷한 경향 |
| 대표 유사도 지표 | 코사인 유사도, 유클리드 거리 | 코사인: 방향(의미) 중심 비교 |
| 장점 | 의미적 관계 수치화, 모델 입력 가능, 일반화에 유리 | 검색/추천/분류/클러스터링 등 |
| 한계 | 데이터·학습 방식에 따라 편향/품질 차이, OOV 문제 | 학습 코퍼스에 없는 단어 처리 |
| 활용 | 분류, 검색, 추천, 군집화, QA, RAG 등 | query↔doc 유사도 계산 |
| 학습 방식(큰 분류) | 정적(Static) vs 문맥(Contextual) | 정적: word2vec / 문맥: BERT 계열 |
| 정적 임베딩 특징 | 단어당 벡터 1개(문맥 변화 반영 어려움) | “bank” (강둑/은행) 구분 약함 |
| 문맥 임베딩 특징 | 같은 단어라도 문맥에 따라 벡터가 달라짐 | “bank”가 문장에 따라 의미 분리 |

## 자연어 임베딩 요약

- **자연어 임베딩(Embedding)**: 텍스트(단어/문장/문서)를 **고정 길이의 실수 벡터**로 변환하는 기술  
- 목적: 컴퓨터가 텍스트를 **수치로 처리**하면서도 **의미/문법 정보**를 벡터에 담게 함  
- 효과:
  - 벡터 간 **유사도(코사인 유사도 등)** 계산 가능 → 관련도/의미 유사성 비교
  - ML/DL 모델의 **입력 피처**로 사용 가능
  - 벡터 공간에서 **의미 관계(유추/연산)** 표현 가능(예: king - man + woman ≈ queen)
- 종류(큰 분류):
  - **정적 임베딩**: 단어당 벡터 1개(문맥 변화 반영 어려움)  
  - **문맥 임베딩**: 같은 단어도 문장 문맥에 따라 벡터가 달라짐